In [ ]:
import itertools as it

In [ ]:
class Node:
    def __init__(self, id, sons=[], dads=[]):
        self.id = id
        self.sons  = sons
        self.dads = dads

    def __str__(self):
        #return "From str method of Node: a is %s, b is %s" % (self.a, self.b)
        return str(self.id)
       # return "test affichage"

In [ ]:
def print_tree(tree):
  if tree == [] :
    print("empty tree")
  else:
    for i in tree:
      print("base : ")
      print(i[0])
      for j in i[1::]:
        print_arbre(j)
        #for x in j:
         #   print(x)


In [ ]:
def print_arbre(L):
  k=0
  decal=0
  for i in L:
    if k==0:
      print(i)
      k=decal+1
      decal = decal+1
    else:
      print(i,end='')
      k=k-1


In [ ]:
 E = [('A',3),('B',4),('C',2),('A',4),('B',2),('C',5)]

# methodes de parcours/ transformation

# methode pour recuperer une liste d'event [A,B,C] a partir de la liste avec les tuples
def get_events(E):
  check_val = set()      #Check Flag
  res = []
  for i in E:
      if i[0] not in check_val:
          res.append(i)
          check_val.add(i[0])
  events = [a_tuple[0] for a_tuple in res]
  return events



In [ ]:
#renvoie base de contraintes pour une paire donnée 

def get_base(E,C,limite_ecarts):
  instance_eventA = []
  instance_eventB = []
  for i in E:
    if C[0]in i :
      instance_eventA.append(i)
    if C[1] in i:
      instance_eventB.append(i)
  ecarts = []
  for x in instance_eventA:
    for y in instance_eventB:
      e = y[1] - x[1]
      #calcul de touts les écarts + tri
      if abs(e) < limite_ecarts:
        ecarts.append(y[1] - x[1])
  tries = sorted(list(set(ecarts)))
  tree = []
  #print(tries)
  if (len(tries) ==1) :
    # renvoyer arbre avec un seul noeud doublé #question: tries de taille 1 : ca veut dire que ya qu'un seul arbre ?? ou ca veut dire qu'il y a un seul écart ??
    tree.append(Node([tries[0],tries[0]],[],[]))
  elif (len(tries) ==2) :
    tree.append(Node([tries[0],tries[1]],[],[]))
  else : 
    arbre(tree,tries)
    
  return tree



In [ ]:
#graph : liste de noeuds ordonnés en largeur d'abord, étage par étage (de gauche à droite, puis de haut en bas)

def not_exists(tree, id):
  found = False
  L = len(tree)
  k = 0
  while k<L and not(found):
    N = tree[k]

    if(N.id == id):
      found = True

    k = k+1
  return not(found)

In [ ]:
#finie
def add_son(tree, id_dad, id_son):
  k = len(tree)
  found = False
  while not(found) and k!=0 :
    if(tree[k-1].id == id_dad):
      tree[k-1].sons.append(id_son)
    k = k-1
  
  

In [ ]:
#finie
def find_dads(tree, id):
  dads = []
  if tree != []:
    found = False
    first_found = False
    k = len(tree)
    while not(found) and k!=0 :
      k = k-1
      id_dad = tree[k].id #maybe
      [x_dad,y_dad] = id_dad
      [x,y] = id

      if(x>=x_dad) and (y<=y_dad):
        dads.append(id_dad)
        add_son(tree, id_dad, id)
        if(first_found):
          found = True
        else:
          first_found = True
      else:
        if(first_found):
          found = True
  return dads

In [ ]:
#finie
def new_node(tree,id):
  if not_exists(tree, id):
    [x,y]=id
    sons = []
    dads = find_dads(tree, id)
    N = Node(id, sons, dads)
    tree.append(N)

In [ ]:
# construit une base de contraintes ordonnée à partir de la liste de tout les écarts

#L : liste des écarts 
# if len(L) == 1 : déjà traité avant !!!
def arbre(tree,L):
  for k in range(len(L),1,-1):
    #pour k = len-1 (premier etage en dessous de la racine)
    # i va de 0 à 1 : deux iterations => 2 fils
    for i in range(0, len(L)-k+1):
      x = L[i]
      y = L[i+k-1]
      paire = [x,y]
      new_node(tree, paire)
  
  
  #arbre tree construit

In [ ]:
def CCDC(E,lim):
  base = []
 #itere sur toutes les paires d'event ordonnés lexi ex : ['A','B']
  events = get_events(E)
  events.sort()
  for comb in it.combinations(events,2) :
    #print(comb)
    B = get_base(E,comb,lim)
    base.append([comb,B])

  return base
    


In [ ]:
def get_comb_gaps(E, events_comb, lim, gaps):
     instance_eventA = []
     instance_eventB = []
     for i in E:
       if events_comb[0] in i :
         instance_eventA.append(i)
       if events_comb[1] in i:
         instance_eventB.append(i)

     for x in instance_eventA:
       for y in instance_eventB:
         e = y[1] - x[1]
         #calcul de touts les écarts + tri
         if abs(e) < lim:
           gaps.append(y[1] - x[1])
     return gaps

In [ ]:
def get_gaps(E, lim, gaps_dict):
    events = get_events(E)
    events.sort()
    for comb in it.combinations(events,2) :
        key = comb[0] + comb[1]
        if not comb in gaps_dict:
            gaps_dict[comb] = get_comb_gaps(E, comb, lim, [])
        else:
            get_comb_gaps(E, comb, lim, gaps_dict[comb])

In [ ]:
def generate_trees(gaps_dict):
    trees = []

    for comb in gaps_dict:
        tries = sorted(list(set(gaps_dict[comb])))
        tree = []
        #print(tries)
        if (len(tries) ==1) :
          # renvoyer arbre avec un seul noeud doublé #question: tries de taille 1 : ca veut dire que ya qu'un seul arbre ?? ou ca veut dire qu'il y a un seul écart ??
          tree.append(Node([tries[0],tries[0]],[],[]))
        elif (len(tries) ==2) :
          tree.append(Node([tries[0],tries[1]],[],[]))
        else :
          arbre(tree,tries)
        trees.append([comb,tree])

    return trees

In [ ]:
base = CCDC(E,50)
print_tree(base)

base : 
('A', 'B')
[-2, 1]
[-2, 0][-1, 1]
[-2, -1][-1, 0][0, 1]
base : 
('A', 'C')
[-2, 2]
[-2, 1][-1, 2]
[-2, -1][-1, 1][1, 2]
base : 
('B', 'C')
[-2, 3]
[-2, 1][0, 3]
[-2, 0][0, 1][1, 3]
